In [16]:
import pandas as pd
df = pd.read_csv('data/canada_2015_2019.csv')
len(df)

596

In [17]:
df = df.loc[df[df.columns[2]] == 2019]
df.head()


,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10
0,0,Canada,2019,Bow River (electoral district),Conservative,Blake Richards,71.1,True,Liberal,Gwyneth Midgley,10.8,False
2,2,Canada,2019,Banff—Airdrie,Conservative,Damien Kurek,85.5,True,New Democratic,Natasha Fryzuk,5.1,False
4,4,Canada,2019,Battle River—Crowfoot,Conservative,Martin Shields,83.9,True,Liberal,Margaret Rhemtulla,5.8,False
6,6,Canada,2019,Bow River (electoral district),Liberal,Kent Hehr,46.52,True,Conservative,Joan Crockatt,45.30,False
7,7,Canada,2019,Calgary Centre,Conservative,Len Webber,55.1,True,Liberal,Jordan Stein,22.6,False


In [18]:
winners = df[[df.columns[4], df.columns[5]]]
losers = df[[df.columns[8], df.columns[9]]]
losers.columns = ['candidate', 'party']
winners.columns = ['candidate', 'party']
all_candidates = pd.concat([winners, losers])
all_candidates
all_candidates['candidate'] = all_candidates['candidate'].astype(str)
all_candidates['party'] = all_candidates['party'].astype(str)
all_candidates = all_candidates.applymap(lambda x: x.strip() if isinstance(x, str) else x)
all_candidates.drop_duplicates(inplace = True, subset = ['candidate', 'party'])
all_candidates                                    

,candidate,party
0,Conservative,Blake Richards
2,Conservative,Damien Kurek
4,Conservative,Martin Shields
6,Liberal,Kent Hehr
7,Conservative,Len Webber
9,Conservative,Jasraj Hallan
11,Conservative,Bob Benzen
13,Conservative,Stephanie Kusie
15,Conservative,Michelle Rempel
17,Conservative,Pat Kelly


In [19]:
all_candidates['candidate'] = all_candidates['candidate']+" "+all_candidates['party']
search_terms = all_candidates['candidate']
search_terms


0                    Conservative Blake Richards
2                      Conservative Damien Kurek
4                    Conservative Martin Shields
6                              Liberal Kent Hehr
7                        Conservative Len Webber
9                     Conservative Jasraj Hallan
11                       Conservative Bob Benzen
13                  Conservative Stephanie Kusie
15                  Conservative Michelle Rempel
17                        Conservative Pat Kelly
19                        Conservative Tom Kmiec
21                      Conservative Ron Liepert
23                       Conservative Jag Sahota
25                    Liberal Randy Boissonnault
26                     Conservative Kerry Diotte
27                   Conservative Ziad Aboultaif
29                    Conservative Matt Jeneroux
30                   Conservative Kelly McCauley
32                        Conservative Mike Lake
34                      Conservative John Barlow
36                  

In [5]:
from googleapiclient.discovery import build
import pprint
import pandas as pd
from PIL import Image
import requests
from io import BytesIO
import cv2
import numpy as np
import os
import time
import requests
from PIL import Image
from io import BytesIO
from keys_file import GOOGLE_KEY
def get_imgs_from_urls(img_urls):
    images = []
    for image_url in img_urls:
            print(image_url)
            try:
                response = requests.get(image_url)
                image = Image.open(BytesIO(response.content))
                images.append(image)
            except: 
                print("ERROR")
                None
    return images

def get_image_from_json(json):
    images = []
    for item in json['value']:
        try:
            response = requests.get(item['contentUrl'])
            image = Image.open(BytesIO(response.content))
            images.append(image)
        except: 
            None
    return images
def count_faces(image):
    xml_files = ['xml_files/haarcascade_frontalface_default.xml','xml_files/lbpcascade_frontalface.xml']
    faces_counted = []
    for xml_file in xml_files:
        face_cascade = cv2.CascadeClassifier(xml_file)
        gray = cv2.cvtColor(np.float32(image), cv2.COLOR_BGR2GRAY)
        gray = np.array(gray, dtype='uint8')
        faces = face_cascade.detectMultiScale(gray, 1.2, 5)
        faces_counted.append(len(faces))
        if len(faces) == 1:
            return 1
            break
    return 0
        
# MTCNN uses depricted functions.
def count_faces_dnn(image):
    img = cv2.cvtColor(np.float32(image), cv2.COLOR_BGR2RGB)
    detector = MTCNN()
    faces = detector.detect_faces(img)
    print(len(faces))    
    return None
        

def check_directory(query):
    directory = 'images canada 2019/'+query
    if not os.path.exists(directory):
        os.makedirs(directory)
    return directory

def select_relevant_images(images):
#   check if url contains portrait
    relevant_images = []
    for image  in images:
        faces_count = 0
        try:
            faces_count = count_faces(image)
        except:
            None
        if faces_count == 1:
            relevant_images.append(image)
    return relevant_images

def write_images_to_directory(directory, images, query):
    counter = 1
    for image in images:
        if not image.mode == 'RGB':
            image = image.convert('RGB')
        image.save(directory+"/google_"+query+str(counter)+".jpeg")
        counter += 1;
        
def get_google_urls(search_term):
    service = build("customsearch", "v1",
            developerKey= GOOGLE_KEY)

    res = service.cse().list(
      q=search_term,
      cx='014171572285050841134:z8jpb8kffjb',
      num='8',
      searchType='image'
#       imgType='face'
    ).execute()
    urls = []
    for item in res['items']:
#         print(item['link'])
        urls.append(item['link'])
    return urls


In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
counter = 0
for search_term in search_terms[627:]:
    print(search_term)
    print(counter, "of", len(search_terms[627:]))
    urls = get_google_urls(search_term)
    print("getting urls")
    if urls:
#         print(urls)
        print("GETTINF IMAGES")
        images = get_imgs_from_urls(urls)
        print("TO DIRECTORY")

        directory = check_directory(search_term)
#         images = select_relevant_images(images)
        write_images_to_directory(directory, images, search_term)
        print("DONE")
    else:
        print('No images found') 
    time.sleep(2)
    counter += 1
print("DONE")

New Democratic Harmony Johnson-Harder
0 of 11
getting urls
GETTINF IMAGES
https://paherald.sk.ca/wp-content/uploads/2019/09/Johnson-Harder.jpg
https://media.socastsrm.com/wordpress/wp-content/blogs.dir/1479/files/2019/09/a9d829e0-50dd-4b51-9510-53b880470673.jpg
https://www.fairobserver.com/wp-content/uploads/2019/12/UK-2-1.jpg
ERROR
https://media.socastsrm.com/wordpress/wp-content/blogs.dir/1479/files/2019/10/ba3c502933d0d95bd7cfa5b67f23b23c87dbf80cb8ab5027a48b503b6a728629.jpg
https://upload.wikimedia.org/wikipedia/commons/b/b0/Canada_Election_2019_Results_Map_%28Simple%29.svg
ERROR
https://media.socastsrm.com/wordpress/wp-content/blogs.dir/1479/files/2019/10/e92bc9f1-d819-4231-94c1-a0c9edbbb5c2.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/b/b0/Canada_Election_2019_Results_Map_%28Simple%29.svg/500px-Canada_Election_2019_Results_Map_%28Simple%29.svg.png
https://media.socastsrm.com/wordpress/wp-content/blogs.dir/1479/files/2019/10/bbfde2d9-faf3-4311-b47e-40dd9f190283.jpg
TO D

In [ ]:
all_candidates.to_csv('data/all_candidates_canada_2019.csv')
# 137 of 348
len(all_candidates)